<a href="https://colab.research.google.com/github/ma23193/Dissertation/blob/main/disser9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
def load_data(noisy_path, clean_path):
    noisy_signals = np.load(noisy_path)
    clean_signals = np.load(clean_path)

    # Ensure the data is in the right shape
    noisy_signals = noisy_signals[..., np.newaxis]
    clean_signals = clean_signals[..., np.newaxis]

    return noisy_signals, clean_signals

noisy_path = 'path/to/your/noisy_signals.npy'
clean_path = 'path/to/your/clean_signals.npy'

noisy_signals, clean_signals = load_data(noisy_path, clean_path)


In [ ]:
def create_dataset(noisy_signals, clean_signals, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((noisy_signals, clean_signals))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

batch_size = 32
dataset = create_dataset(noisy_signals, clean_signals, batch_size)


In [ ]:
def build_generator(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    e1 = layers.Conv1D(512, 1, padding='same', activation='linear')(inputs)
    e1 = layers.PReLU()(e1)
    e2 = layers.Conv1D(512, 16, padding='same', activation='linear')(e1)
    e2 = layers.PReLU()(e2)
    e3 = layers.Conv1D(512, 32, padding='same', activation='linear')(e2)
    e3 = layers.PReLU()(e3)
    e4 = layers.Conv1D(512, 64, padding='same', activation='linear')(e3)
    e4 = layers.PReLU()(e4)
    e5 = layers.Conv1D(256, 128, padding='same', activation='linear')(e4)
    e5 = layers.PReLU()(e5)
    e6 = layers.Conv1D(128, 256, padding='same', activation='linear')(e5)
    e6 = layers.PReLU()(e6)
    e7 = layers.Conv1D(64, 512, padding='same', activation='linear')(e6)
    e7 = layers.PReLU()(e7)
    e8 = layers.Conv1D(32, 1024, padding='same', activation='linear')(e7)
    e8 = layers.PReLU()(e8)

    # Decoder with skip connections
    d7 = layers.Conv1DTranspose(64, 512, padding='same', activation='linear')(e8)
    d7 = layers.PReLU()(d7)
    d7 = layers.Concatenate()([d7, e7])
    d6 = layers.Conv1DTranspose(128, 256, padding='same', activation='linear')(d7)
    d6 = layers.PReLU()(d6)
    d6 = layers.Concatenate()([d6, e6])
    d5 = layers.Conv1DTranspose(256, 128, padding='same', activation='linear')(d6)
    d5 = layers.PReLU()(d5)
    d5 = layers.Concatenate()([d5, e5])
    d4 = layers.Conv1DTranspose(512, 64, padding='same', activation='linear')(d5)
    d4 = layers.PReLU()(d4)
    d4 = layers.Concatenate()([d4, e4])
    d3 = layers.Conv1DTranspose(512, 32, padding='same', activation='linear')(d4)
    d3 = layers.PReLU()(d3)
    d3 = layers.Concatenate()([d3, e3])
    d2 = layers.Conv1DTranspose(512, 16, padding='same', activation='linear')(d3)
    d2 = layers.PReLU()(d2)
    d2 = layers.Concatenate()([d2, e2])
    d1 = layers.Conv1DTranspose(512, 1, padding='same', activation='linear')(d2)
    d1 = layers.PReLU()(d1)
    d1 = layers.Concatenate()([d1, e1])

    outputs = layers.Conv1DTranspose(1, 1, padding='same', activation='linear')(d1)

    model = models.Model(inputs, outputs, name='generator')
    return model

generator = build_generator((None, 1))
generator.summary()


In [ ]:
def build_discriminator(input_shape):
    inputs = layers.Input(shape=input_shape)

    d = layers.Conv1D(64, 5, strides=2, padding='same')(inputs)
    d = layers.BatchNormalization()(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv1D(128, 5, strides=2, padding='same')(d)
    d = layers.BatchNormalization()(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv1D(256, 5, strides=2, padding='same')(d)
    d = layers.BatchNormalization()(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv1D(512, 5, strides=2, padding='same')(d)
    d = layers.BatchNormalization()(d)
    d = layers.LeakyReLU(alpha=0.2)(d)

    d = layers.Flatten()(d)
    d = layers.Dense(1, activation='sigmoid')(d)

    model = models.Model(inputs, d, name='discriminator')
    return model

discriminator = build_discriminator((None, 2))
discriminator.summary()


In [ ]:
def generator_loss(fake_output, real_signal, generated_signal):
    ldist = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(generated_signal - real_signal), axis=1)))
    lmax = tf.reduce_max(tf.abs(generated_signal - real_signal))
    lsgan_loss = tf.reduce_mean(tf.square(fake_output - 1))
    return lsgan_loss + 0.7 * ldist + 0.2 * lmax

def discriminator_loss(real_output, fake_output):
    real_loss = tf.reduce_mean(tf.square(real_output - 1))
    fake_loss = tf.reduce_mean(tf.square(fake_output))
    return real_loss + fake_loss


In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(noisy_signals, real_signals):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_signals = generator(noisy_signals, training=True)

        real_output = discriminator(tf.concat([real_signals, noisy_signals], axis=2), training=True)
        fake_output = discriminator(tf.concat([generated_signals, noisy_signals], axis=2), training=True)

        gen_loss = generator_loss(fake_output, real_signals, generated_signals)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

def train(dataset, epochs):
    for epoch in range(epochs):
        for noisy_signals, real_signals in dataset:
            gen_loss, disc_loss = train_step(noisy_signals, real_signals)
        print(f'Epoch {epoch+1}, Gen Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}')
